In [1]:
import logging
logging.basicConfig(filename='etl.log', level=logging.INFO,
 format='%(asctime)s:%(levelname)s:%(message)s')

In [10]:
import pandas as pd
import logging
from sqlalchemy import create_engine

try:
    # Configurar la conexión con SQL Server usando autenticación de Windows
    server = 'localhost' # Cambiar si es necesario
    database = 'COVID19'
    driver = 'SQL Server Native Client 11.0'

    connection_string = f'mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes'
    engine = create_engine(connection_string)

    # Leer la consulta SQL desde un archivo
    with open('query.sql') as file:
        query = file.read()

    # Extraer los datos de la tabla StagingPatients
    data = pd.read_sql(query, engine)

    logging.info('Datos extraidos exitosamente de SQL Server.')
except Exception as e:
    logging.error('Error extrayendo datos: {e}')
    raise 

try:
    # Convertir la columna 'WeekEnd' a tipo datetime
    data['WeekEnd'] = pd.to_datetime(data['WeekEnd'])

    # Convertir valores nulos en las culumnas a 0
    data.fillna(0, inplace=True)

    logging.info('Datos transformados exitosamente')
except Exception as e:
    logging.error('Error transformando datos: {e}')
    raise 

try:
    # Guardar los datos transformados en una tabla en SQL Server
    data.to_sql('Patients', engine, index=False, if_exists='append')

    logging.info('Datos cargados exitosamente a SQL Server')
except:
    logging.error('Error cargando datos a SQL Server: {e}')
    raise




In [21]:
import pandas as pd
import pyodbc
import os

# Ejecutar el script SQL para transformar e insertar los datos
try:
    # Conectar a la base de datos SQL Server (base de datos target)
    server = 'localhost' # Cambiar si es necesario
    driver = 'SQL Server Native Client 11.0'
    database_target = 'DimensionalDB'
    
    conn_target = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database_target};Trusted_Connection=yes;')
    cursor_target = conn_target.cursor()

    # Ruta al archivo SQL
    sql_file_path = 'dimensional_model.sql'

    # Verificar si el archivo SQL existe y ejecutarlo
    if os.path.exists(sql_file_path):
        with open(sql_file_path, 'r') as sql_file:
            sql_commands = sql_file.read()
        
        try:
            # Ejecutar los comandos SQL de truncamiento y creación de tablas
            cursor_target.execute(sql_commands)
            conn_target.commit()
            logging.info("Script SQL cargar datos a Fact y Dimensionales.")
        except Exception as e:
            logging.error(f"Error al ejecutar el  SQL: {e}")
            conn_target.rollback()
    else:
        logging.error(f"El archivo SQL '{sql_file_path}' no existe")

finally:
    # Cerrar la conexión a la base de datos de destino
    cursor_target.close()
    conn_target.close()
